In [ ]:
using GuSTO
using AstrobeeRobot

In [ ]:
robot = Car()
model = DubinsCar()
env = BlankEnv()

N = 50
x_init = 2*ones(3)
x_goal = zeros(3)
tf_guess = 5.

PD = ProblemDefinition(robot, model, env, x_init, x_goal)

TOP = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)

In [ ]:
# SCP Only
TOS_SCP = TrajectoryOptimizationSolution(TOP)
solve_SCP!(TOS_SCP, TOP, solve_gusto_cvx!, init_traj_straightline, "Gurobi", OutputFlag=0)
# solve_SCP!(TOS_SCP, TOP, solve_gusto_jump!, init_traj_straightline, "IPOPT", print_level=0)

@show TOS_SCP.SCPS.successful
@show TOS_SCP.SCPS.converged
@show TOS_SCP.SCPS.iterations
@show TOS_SCP.SCPS.total_time
@show TOS_SCP.SCPS.J_true[end]

In [ ]:
@show TOS_SCP.SCPS.successful
@show TOS_SCP.SCPS.converged
@show TOS_SCP.SCPS.iterations
@show TOS_SCP.SCPS.iter_elapsed_times
@show TOS_SCP.SCPS.total_time
@show TOS_SCP.SCPS.accept_solution
@show TOS_SCP.SCPS.prob_status
@show TOS_SCP.SCPS.convergence_measure
@show TOS_SCP.SCPS.param.alg.omega_vec
@show TOS_SCP.SCPS.param.alg.Delta_vec

In [ ]:
TOS_SCPS  = TrajectoryOptimizationSolution(TOP)
solve_SCPshooting!(TOS_SCPS, TOP, solve_gusto_jump!, init_traj_straightline, "IPOPT", print_level=0)

@show TOS_SCPS.SS.converged
@show TOS_SCPS.SCPS.total_time

In [ ]:
using PyPlot
plot(TOS_SCPS.traj.X[1,:], TOS_SCPS.traj.X[2,:])
xlabel("x")
ylabel("y")
axis("equal")